In [2]:
!which python

/Users/petergunn/opt/anaconda3/envs/NewPythonData/bin/python


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "./cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [ ]:
from citipy import citipy

In [9]:
!which citipy

# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
## Generate Cities List
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

NameError: name 'citipy' is not defined

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Base URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'

# List to hold response info
city_list = []


i = 0
set_count = 1
for index, city in enumerate(cities):
    i +=1
    query_url = url +'appid=' + weather_api_key +'&q='+ city + '&units=' + units
    response = requests.get(query_url).json()
    #print(response)
    print(f'Loading .. {index} of {set_count} | {city}') 
    if response['cod'] == '404':
        print(f'Could not find city. Pending...')
    else:
        city_list.append({
            'City': response['name'],
            'Lat': response['coord']['lat'],
            'Lng': response['coord']['lon'],
            'Max Temp': response['main']['temp_max'],
            'Humidity': response['main']['humidity'],
            'Cloudiness': response['clouds']['all'],
            'Wind Speed': response['wind']['speed'],
            'Country': response['sys']['country'],
            'Date': response['dt']
             })
        
    if i == 49:
        i =0
        set_count +=1       

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_data = pd.DataFrame(city_list)
weather_data

# Export the city data into a .csv.
weather_data.to_csv('./cities.csv', encoding='utf-8', index=False)

# Display the DataFrame
weather_data

# Finding cities with humidity over 100%

In [ ]:
humidity_hundred = (len(weather_data.index[weather_data['Humidity']>100]))
weather_data.describe()

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#Plot latitude vs temperature 
plt.scatter(weather_data['Lat'], weather_data['Max Temp'], marker='o', alpha=0.5, edgecolors='black')

plt.title('City Latitude vs Max Temperature')
plt.ylabel('Max Temperature(F)')
plt.xlabel('Latitude')
plt.grid(True)
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
#Plot latitude vs Humidity 

plt.scatter(weather_data['Lat'], weather_data['Humidity'], marker='o', alpha=0.5, edgecolors='black')

plt.title('City Latitude vs Humidity')
plt.ylabel('Humidity(%)')
plt.xlabel('Latitude')
plt.grid(True)
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
#Plot latitude vs Cloudiness 
plt.scatter(weather_data['Lat'], weather_data['Cloudiness'], marker='o',alpha=0.5, edgecolors='black')

plt.title('City Latitude vs Cloudiness ')
plt.ylabel('Cloudiness(%)')
plt.xlabel('Latitude')
plt.grid(True)
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
#Plot latitude vs Wind_Speed 
plt.scatter(weather_data['Lat'], weather_data['Wind Speed'], marker='o', alpha=0.5, edgecolors='black')

plt.title('City Latitude vs Wind Speed')
plt.ylabel('Wind Speed')
plt.xlabel('Latitude')
plt.grid(True)
plt.show()

## Linear Regression

In [3]:
#North Hemisphese citiess 
north_hemi = weather_data.loc[weather_data['Lat'] >= 0]
north_hemi_cities = len(north_hemi)

print(f'There are {north_hemi_cities} cities in the Northern Hemisphere.')
north_hemi


NameError: name 'weather_data' is not defined

In [ ]:
Southern Hemisphere cities
south_hemi = weather_data.loc[weather_data['Lat']<0]
south_hemi_cities = len(south_hemi)

print(f'There are {south_hemi_cities} cities in the Southern Hemisphere.')
south_hemi

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi['Lat']
y_values = north_hemi['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
line_eq

print(f'r-value: {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Northern Hemisphere Cities Latitude vs Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature(F)')
plt.annotate(line_eq,(5,5), fontsize=15, color='black')



plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi['Lat']
y_values = south_hemi['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f' r-value: {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Southern Hemisphere Cities Latitude vs Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temp(F)')
plt.annotate(line_eq,(-25,45), fontsize=15, color='black')


plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi['Lat']
y_values = north_hemi['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

# Print r value
print(f'r-value: {round(rvalue**2, 2)}')

# Plot Linea Regression
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Northern Hemisphere Cities Latitude vs Humidity ')
plt.xlabel('Latitude')
plt.ylabel('Humidity(%)')
plt.annotate(line_eq,(45,1), fontsize=15, color='black')

# Save the plotted figure as .png
plt.savefig('../output_data/north_lat_vs_humidity.png')

# Show the plot
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi['Lat']
y_values = south_hemi['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f'r-value : {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Southern Hemisphere Cities Latitude vs Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity(%)')
plt.annotate(line_eq,(-56,50), fontsize=14, color='black')


plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi['Lat']
y_values = north_hemi['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f'r-value: {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Northern Hemisphere Cities Latitude vs Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness(%)')
plt.annotate(line_eq,(40, 47), fontsize=15, color='black')


plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi['Lat']
y_values = south_hemi['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f' r-value : {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Southern Hemisphere Cities Latitude vs Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness(%)')
plt.annotate(line_eq,(-50,55), fontsize=15, color='black')


plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi['Lat']
y_values = north_hemi['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f' r-value : {round(rvalue**2, 3)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Northern Hemisphere Cities Latitude vs Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.annotate(line_eq,(10,25), fontsize=15, color='black')


plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi['Lat']
y_values = south_hemi['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = 'y = ' + str(round(slope,2)) + 'x + ' + str(round(intercept,2))
print(line_eq)

print(f' r-value : {round(rvalue**2, 2)}')

plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,'r-')
plt.title('Southern Hemisphere Cities Latitude vs Wind Speed ')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed(MPH)')
plt.annotate(line_eq,(-55, 12), fontsize=15, color='red')


plt.show()